# Model 1: 3 Convolutions then 3 Deconvolutions

## Parametres

In [1]:
#Depencies
import os
from random import shuffle
from math import floor
from PIL import Image

In [2]:
#Create a CNN
import numpy as np
from keras.layers import Dense, Dropout, Conv2D, Flatten, Input
from keras.layers import Reshape, Conv2DTranspose, MaxPooling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.utils import plot_model
from keras.backend import int_shape
from keras.models import load_model

Using TensorFlow backend.


In [3]:
#Set WD
os.chdir('C:/Users/feher/Documents/ScriptDeepLearningColorPhoto/')

In [4]:
#Data's path
pathColor = "D:/Project/DeepLearning/AlgoPhotoColor/Data/color/"
pathNb = "D:/Project/DeepLearning/AlgoPhotoColor/Data/nb/"

In [5]:
#Parametres
h,w  = (200,200)
filters = [32,64,128]
latentVectLen = 256
strides1 = 2
kernel1 = 3

In [6]:
#Parametres
batchSize = 20
epoch = 1

## Model Creation

In [7]:

#Autoencoder

###Encoder part
inputs = Input(  name = 'encoder_input',shape=(h,w,1) )
x = inputs
index = 0

for f in filters:
    x = Conv2D(filters= f , kernel_size = kernel1, activation='relu', padding='same', strides= strides1  )(x)
    x = Dropout(0.2)(x)
    index += 1

shp = int_shape(x)#Get the shape of the vector
x = Flatten()(x)
x = Dense(latentVectLen)(x)#Latent vector

###Decoder part

x = Dense( shp[1]*shp[2]*shp[3] )(x)
x = Reshape((shp[1],shp[2],shp[3]))(x)

index = 2
for f in filters[::-1]:
    x = Conv2DTranspose(filters=f, kernel_size=kernel1,  activation='relu', padding="same", strides=strides1)(x)
    x = Dropout(0.2)(x)
    index += -1

outputs = Conv2DTranspose(filters=3, kernel_size=kernel1, activation='sigmoid',padding='same')(x)


autoEnc = Model(inputs, outputs, name='autoEnc')

In [8]:
autoEnc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 200, 200, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 128)       73856     
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 25, 128)       0         
__________

In [9]:
autoEnc.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

## Load datas

In [10]:
#Loading train Data
Xtr = []
Ytr = []
file = open("D:/Project/DeepLearning/AlgoPhotoColor/Data/train.txt")
fileTr = file.readlines()
file.close()


for f in fileTr:
    Ytr.append( np.load(pathColor + f[:-1]))
    Xtr.append( np.load(pathNb + f[:-1]))

#Loading test data
Xtes = []
Ytes = []
file = open("D:/Project/DeepLearning/AlgoPhotoColor/Data/test.txt")
fileTes = file.readlines()
file.close()

for f in fileTes:
    Ytes.append( np.load(pathColor + f[:-1]))
    Xtes.append( np.load(pathNb + f[:-1]))

In [11]:
# reshape
Xtr = np.asarray(Xtr).flatten()
Xtr = Xtr.reshape(-1,w,h,1)
Ytr = np.asarray(Ytr).flatten()
Ytr = Ytr.reshape((-1,w,h,3))

Xtes = np.asarray(Xtes).flatten()
Xtes = Xtes.reshape(-1,w,h,1)
Ytes = np.asarray(Ytes).flatten()
Ytes = Ytes.reshape((-1,w,h,3))

## view a sample of picture

In [ ]:
index = 0
for pics in Ytes[0:20]:
    z = pics * 255
    z=Image.fromarray(z.astype('uint8'))
    z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/test/original{}.png'.format(index))
    z = z.convert('L')
    z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/test/original{}_n.png'.format(index))
    index+= 1
    
index = 0
for pics in Ytr[0:20]:
    z = pics * 255
    z=Image.fromarray(z.astype('uint8'))
    z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/train/original{}.png'.format(index))
    z = z.convert('L')
    z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/train/original{}_n.png'.format(index))
    index+= 1

## Model Training

In [ ]:
tbd = TensorBoard("D:/Project/DeepLearning/AlgoPhotoColor/outputs/Log/Model2")

In [ ]:
step = 10
epochStart = 30
epochEnd = 50

In [ ]:
#Loop through epoch and very step epochs save the model, and genrate some picture

for ep in range(epochStart,epochEnd,step):
    
    #===============Train the model=======================================
    autoEnc.fit(Xtr,Ytr,batch_size=15,callbacks=[tbd],validation_data=(Xtes,Ytes) ,initial_epoch=ep ,epochs= ep+step)
    
    #==================Save the model=========================================
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/models/Model2/Epoch{}'.format(ep + step))#create the folder
    autoEnc.save( "D:/Project/DeepLearning/AlgoPhotoColor/models/Model2/Epoch{}/autoEnc2{}.h5".format(ep + step,ep + step))
    
    #==================Create output data=====================================
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}'.format(ep + step))
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/test'.format(ep + step))
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/train'.format(ep + step))
    
    
    #export the genrated pictures (test)
    res = autoEnc.predict(Xtes[0:20])
    res *= 255
    res = res.astype('uint8')

    index = 1
    for img in res:
        z = Image.fromarray( img )
        z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/test/Photo{}.png'.format(ep + step,index))
        index += 1
    
    #export the genrated pictures (train)
    res = autoEnc.predict(Xtr[0:20])
    res *= 255
    res = res.astype('uint8')
    index = 1
    for img in res:
        z = Image.fromarray( img )
        z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/train/Photo{}.png'.format(ep + step,index))
        index += 1
    